# GPT2 Text Generation with KerasNLP:

- Before we begin
- Install KerasNLP, Choose Backend and Import Dependencies
- Introduction to Generative Large Language Models (LLMs)
- Introduction to KerasNLP
- Load a pre-trained GPT-2 model and generate some text
- More on the GPT-2 model from KerasNLP
- Finetune on Reddit dataset
- Into the Sampling Method
- Finetune on Chinese Poem Dataset

# Before we begin

In this tutorial:
1. You will learn to use [KerasNLP](https://keras.io/keras_nlp/) to load a pre-trained Large Language Model (LLM) - [GPT-2 model](https://openai.com/research/better-language-models) (originally invented by OpenAI)
2. finetune it to a specific text style
3. generate text based on users' input (also known as prompt). You will also learn how GPT2 adapts quickly to non-English languages, such as Chinese.

This tutorial is demonstrated on OpenShift with advanced features, including:
- Dev Spaces IDE with Jupyter
- shareable notebook images
- node autoscaling
- distributed training on GPUs
- multi-instance GPUs

# Install KerasNLP, Choose Backend and Import Dependencies

This examples uses [Keras Core](https://keras.io/keras_core/) to work in any of "tensorflow", "jax" or "torch". Support for Keras Core is baked into KerasNLP, simply change the "KERAS_BACKEND" environment variable to select the backend of your choice. We select the JAX backend below.

Source tutorial: https://keras.io/examples/generative/gpt2_text_generation_with_kerasnlp/

In [1]:
%pip install pip -U -q
%pip install tensorflow~=2.13.1 keras-nlp==0.6.2 -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow" # "jax"  # or "tensorflow" or "torch"

import keras_nlp
import tensorflow as tf
import keras_core as keras
import time

Using TensorFlow backend


2023-10-17 16:28:50.470810: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-17 16:28:50.533828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 16:28:51.337272: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Introduction to Generative Large Language Models (LLMs)

Large language models (LLMs) are a type of machine learning models that are trained on a large corpus of text data to generate outputs for various natural language processing (NLP) tasks, such as text generation, question answering, and machine translation.

Generative LLMs are typically based on deep learning neural networks, such as the [Transformer architecture](https://arxiv.org/abs/1706.03762) invented by Google researchers in 2017, and are trained on massive amounts of text data, often involving billions of words. These models, such as Google [LaMDA](https://blog.google/technology/ai/lamda/) and [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html), are trained with a large dataset from various data sources which allows them to generate output for many tasks. The core of Generative LLMs is predicting the next word in a sentence, often referred as Causal LM Pretraining. In this way LLMs can generate coherent text based on user prompts. For a more pedagogical discussion on language models, you can refer to the [Stanford CS324 LLM class](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

# Introduction to KerasNLP

Large Language Models are complex to build and expensive to train from scratch. Luckily there are pretrained LLMs available for use right away. [KerasNLP](https://keras.io/keras_nlp/) provides a large number of pre-trained checkpoints that allow you to experiment with SOTA models without needing to train them yourself.

KerasNLP is a natural language processing library that supports users through their entire development cycle. KerasNLP offers both pretrained models and modularized building blocks, so developers could easily reuse pretrained models or stack their own LLM.

In a nutshell, for generative LLM, KerasNLP offers:

Pretrained models with generate() method, e.g., [keras_nlp.models.GPT2CausalLM](https://keras.io/api/keras_nlp/models/gpt2/gpt2_causal_lm#gpt2causallm-class) and [keras_nlp.models.OPTCausalLM](https://keras.io/api/keras_nlp/models/opt/opt_causal_lm#optcausallm-class).
Sampler class that implements generation algorithms such as Top-K, Beam and contrastive search. These samplers can be used to generate text with custom models.

# Load a pre-trained GPT-2 model and generate some text

KerasNLP provides a number of pre-trained models, such as [Google Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html) and [GPT-2](https://openai.com/research/better-language-models). You can see the list of models available in the [KerasNLP repository](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

It's very easy to load the GPT-2 model as you can see below:

In [3]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

1042301/1042301 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step   
456318/456318 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step     
497986112/497986112 ━━━━━━━━━━━━━━━━━━━━ 21s 0us/step


Once the model is loaded, you can use it to generate some text right away. Run the cells below to give it a try. It's as simple as calling a single function generate():

In [4]:
start = time.time()

# We slightly altered the text to understand what the model thought of us?
output = gpt2_lm.generate("Red hat is ", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")

2023-10-17 16:29:50.403363: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5590ebec6010 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-17 16:29:50.403414: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-10-17 16:29:50.516691: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-17 16:29:57.745757: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



GPT-2 output:
Red hat is  a very common and popular type of red hat, but it is also very common in the United States and Canada. The name Red hat is also used to mean the red hat that comes with your hat, and is often associated with a red, white, or blue sweater. Red hat is also a popular choice in the United States and Canada. It is also a common name in the United States and Canada. The name Red hat is also a popular choice in the United States and Canada.
Red hat is often associated with the white hat, and is a common name for any color in the United States and Canada. It is usually associated with a red hat, but can be associated with a blue, brown or green sweater as well. This name is also used to indicate a white hat, but it is also a common name in the United States and Canada.
Red hat is a popular choice in the United States and Canada. It is usually associated with a red
TOTAL TIME ELAPSED: 67.30s


Try another one: **UPDATE THE TEXT IN QUOTES WITH YOUR REQUEST**

In [5]:
start = time.time()

output = gpt2_lm.generate("The answer to the universe and everything is ", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
The answer to the universe and everything is  a lie and a lie, not a lie.
The truth of the universe is  a lie and lie, not a lie. The truth of the world is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the universe is  a lie and lie, not a lie. The truth of the world is  
TOTAL TIME ELAPSED: 26.50s


Notice how much faster the second call is. This is because the computational graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via fine-tuning.

# Save the pre-trained GPT-2 model to object storage

You can save the model in different formats depending on how you intend to serve the model. In short, this save will enable us to do early online experimentation with the pre-trained model.

In [6]:
gpt2_lm.save('../models/gpt2_lm_v1.keras')
#gpt2_lm.save('../models/gpt2_lm.tf')
#gpt2_lm.save('../models/gpt2_lm.h5')